In [1]:
import os
import sys

#Import config file. Update config.py according to your environment
import config

import pandas as pd
import numpy as np

import tensorflow as tf

from Rakuten_preprocessing import Rakuten_img_path

from src.text.classifiers import TFbertClassifier
from src.image.classifiers import ImgClassifier
from src.multimodal.classifiers import TFmultiClassifier

from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

import datetime


2024-03-11 22:32:33.277084: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 22:32:33.322352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 22:32:33.322377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 22:32:33.323761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 22:32:33.332210: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)

#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')


In [3]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Creating train and test sets

In [4]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

## CNN benchmarks

In [5]:
#Name of the summary csv file to save results to
result_file_name = 'results_benchmark_img.csv'

#type of modality
modality = 'image'

#Type of classifier
class_type = 'ImgClassifier'

#training parameters (or list of parameters for gridsearchCV)
num_class = 27
img_size = (224, 224, 3)
n_epochs = 8
batch_size = 32
drop_rate = 0.2
lr0 = 5e-5
lr_min = 1e-6
lr_decay_rate = 0.8

#defining callbacks
callbacks = []
#adding earlystopping callback
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))
#Adding tensorboard callback as the last one
callbacks.append(('TensorBoard', {'log_dir': np.nan, 'histogram_freq': 1, 'update_freq': 'epoch'}))

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#network to test
base_name_list = ['vit_b16', 'ResNet101', 'ResNet50', 'EfficientNetB1', 'vit_b16', 'ResNet152']

#Initializing the list of parameters to batch over
params_list = []

for base_name in base_name_list:
  #Adjusting tensorboard log directory
  log_dir = os.path.join(config.path_to_tflogs, base_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  callbacks[-1][1]['log_dir'] = log_dir
  #adding the set of parameters to the list
  params_list.append({'modality': modality,
                      'class': class_type,
                      'base_name': base_name, 
                      'param_grid': {'img_size': img_size, 'num_class': num_class, 'drop_rate': drop_rate, 
                                    'epochs': n_epochs, 'batch_size': batch_size, 
                                    'learning_rate':lr0, 'lr_decay_rate': lr_decay_rate, 'lr_min': lr_min,
                                    'validation_data': (X_test, y_test), 'callbacks': [callbacks], 'parallel_gpu': False},
                      'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                    })

#Running the batch over params_list
results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = result_file_name)

Fitting:  vit_b16 None
{'img_size': [(224, 224, 3)], 'num_class': [27], 'drop_rate': [0.2], 'epochs': [1], 'batch_size': [32], 'learning_rate': [5e-05], 'lr_decay_rate': [0.8], 'lr_min': [1e-06], 'validation_data': [(                                                tokens  \
0    bas de filles pour les enfants enfants collant...   
1    cosmic planète series en peluche avec capuche ...   
2    dolphin - robot électrique de piscine fond par...   
3                haydaim  14/114 pokemon noir et blanc   
4    lot 4 livres partitions piano bach (busoni) - ...   
..                                                 ...   
995  barbie hot skatin ken doll – roller et patinag...   
996  elephent bébé teether silicone sucette sucette...   
997  piscine veta ronde décoration bois 6.40m x 1.2...   
998  griottines - coffret de 5 cl griottines - coff...   
999  jeu console portable 35 pouces écran 26 retro ...   

                                              img_path  
0    /home/jul/DST/Rakuten/Da

2024-03-11 21:57:40.360533: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 21:57:40.360935: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 21:57:40.420109: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 21:57:40.420207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 21:57:40.420275: I external/local_xla/xla/stream_executor

Found 1000 validated image filenames.
Found 10000 validated image filenames.


2024-03-11 21:57:50.278057: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 21:58:18.076337: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-11 21:58:20.671211: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-11 21:58:22.952547: I external/local_xla/xla/service/service.cc:168] XLA service 0x6f7fa220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-11 21:58:22.952597: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-11 21:58:22.952605: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-11 21:58:22.958616: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disablin

313/313 [==============================] - 238s 597ms/step - loss: 3.2455 - accuracy: 0.2304 - val_loss: 2.2873 - val_accuracy: 0.3660 - lr: 5.0000e-05
Found 1000 validated image filenames.
32/32 [==============================] - 11s 253ms/step
Test set, f1score:  0.3609029242808815
Fitting:  ResNet50 None
{'img_size': [(224, 224, 3)], 'num_class': [27], 'drop_rate': [0.2], 'epochs': [1], 'batch_size': [32], 'learning_rate': [5e-05], 'lr_decay_rate': [0.8], 'lr_min': [1e-06], 'validation_data': [(                                                tokens  \
0    bas de filles pour les enfants enfants collant...   
1    cosmic planète series en peluche avec capuche ...   
2    dolphin - robot électrique de piscine fond par...   
3                haydaim  14/114 pokemon noir et blanc   
4    lot 4 livres partitions piano bach (busoni) - ...   
..                                                 ...   
995  barbie hot skatin ken doll – roller et patinag...   
996  elephent bébé teether silico

2024-03-11 22:08:43.538517: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/img_base_layers/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


313/313 [==============================] - 177s 379ms/step - loss: 2.9051 - accuracy: 0.2114 - val_loss: 2.3963 - val_accuracy: 0.3680 - lr: 5.0000e-05
Found 1000 validated image filenames.
32/32 [==============================] - 5s 85ms/step
Test set, f1score:  0.286039867637365
Fitting:  vit_b16 None
{'img_size': [(224, 224, 3)], 'num_class': [27], 'drop_rate': [0.2], 'epochs': [1], 'batch_size': [32], 'learning_rate': [5e-05], 'lr_decay_rate': [0.8], 'lr_min': [1e-06], 'validation_data': [(                                                tokens  \
0    bas de filles pour les enfants enfants collant...   
1    cosmic planète series en peluche avec capuche ...   
2    dolphin - robot électrique de piscine fond par...   
3                haydaim  14/114 pokemon noir et blanc   
4    lot 4 livres partitions piano bach (busoni) - ...   
..                                                 ...   
995  barbie hot skatin ken doll – roller et patinag...   
996  elephent bébé teether silicone s

/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning:

Resizing position embeddings from 24, 24 to 14, 14



Found 1000 validated image filenames.
Found 10000 validated image filenames.
313/313 [==============================] - 224s 594ms/step - loss: 3.2621 - accuracy: 0.1988 - val_loss: 2.4941 - val_accuracy: 0.2930 - lr: 5.0000e-05
Found 1000 validated image filenames.
32/32 [==============================] - 11s 246ms/step
Test set, f1score:  0.25891129479182295
Fitting:  ResNet152 None
{'img_size': [(224, 224, 3)], 'num_class': [27], 'drop_rate': [0.2], 'epochs': [1], 'batch_size': [32], 'learning_rate': [5e-05], 'lr_decay_rate': [0.8], 'lr_min': [1e-06], 'validation_data': [(                                                tokens  \
0    bas de filles pour les enfants enfants collant...   
1    cosmic planète series en peluche avec capuche ...   
2    dolphin - robot électrique de piscine fond par...   
3                haydaim  14/114 pokemon noir et blanc   
4    lot 4 livres partitions piano bach (busoni) - ...   
..                                                 ...   
995  barbie 

## Fetch and check the saved result file

In [6]:
results = pd.read_csv(os.path.join(config.path_to_results, result_file_name), index_col=0)
results.head()

,modality,class,vectorization,classifier,tested_params,best_params,score_test,score_cv_test,score_cv_train,fit_cv_time,model_path
0,image,ImgClassifier,NaN,VGG16,"{'img_size': [(224, 224, 3)], 'num_class': [27...",NaN,0.443956,NaN,NaN,NaN,image/VGG16


## Visualize tensorflow logs in tensorboard

In [8]:
log_path = os.path.join(config.path_to_tflogs, 'VGG16')

# Ensure the log_path is quoted to handle spaces
quoted_log_path = f'"{log_path}"'

%reload_ext tensorboard
%tensorboard --logdir {quoted_log_path}

Reusing TensorBoard on port 6006 (pid 3045096), started 0:00:04 ago. (Use '!kill 3045096' to kill it.)

## Example usage

In [5]:
#defining callbacks
callbacks = []
def lrscheduler(epoch, lr):
  return max(1e-6, lr * 0.8)
callbacks.append(('LearningRateScheduler', {'schedule': lrscheduler}))
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1, }))

clf = ImgClassifier(base_name='ResNet101', img_size=(224, 224, 3), num_class=27, drop_rate=0.2, epochs=5, batch_size=32, 
                    validation_data=(X_test, y_test), learning_rate=5e-5, callbacks=callbacks)

clf.fit(X_train, y_train)
clf.classification_score(X_test, y_test)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-03-10 13:07:20.850430: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.850632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948545: I external/local_xla/xla/stream_executor

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu